This file should only be used to contain the code used to download data by using the selenium 

and

Upload data to minio

In [1]:
%pip install boto3
# Libraries maybe you need to install in your environment to use selenium and chromedriver below
%pip install selenium chromedriver-binary

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [2]:
import os
import boto3

In [3]:
# The file download processing section should be placed here

#...........................................................

import time
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

options = webdriver.ChromeOptions()
preferences = {
    "download.default_directory": os.getcwd(),
    "download.prompt_for_download": False,
    "download.directory_upgrade": True,
    "safebrowsing.disable_download_protection": True
}
options.add_experimental_option("prefs", preferences)
driver = webdriver.Chrome(options=options)

try:
    driver.get("https://drive.google.com/drive/folders/1EImvKyUQ_iJCfd2h-Dxjn_sqPyDV5tFC")

    download_button = WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.XPATH, '//*[@id=":1"]/div/c-wiz/div/c-wiz/div[1]/c-wiz/div[2]/c-wiz/div[1]/c-wiz/c-wiz/div/c-wiz/div/div/div/div[6]'))
            )
    download_button.click()
    time.sleep(10)
    print(f"File 'data.zip' downloaded successfully")


except Exception as e:
        print(f"An error occurred while downloading the file: {e}")
finally:
    # Close the browser window
    driver.quit()

File 'data.zip' downloaded successfully


In [4]:
import zipfile
with zipfile.ZipFile('./data.zip', 'r') as zip_ref:
    zip_ref.extractall('./')

In [5]:
# Define S3 storage
obj_storage_access_key = os.getenv('OBJ_STORAGE_ACCESS_KEY', 'minioadmin')
obj_storage_secret_key = os.getenv('OBJ_STORAGE_SECRET_KEY', 'minioadmin')
obj_storage_endpoint = os.getenv('OBJ_STORAGE_ENDPOINT', 'http://localhost:9000')


In [6]:
# Function to upload file to minio
def upload_file_to_minio(file_path, minio_bucket, minio_object_name):
    s3c = boto3.resource('s3',
                        endpoint_url=obj_storage_endpoint,
                        aws_access_key_id=obj_storage_access_key,
                        aws_secret_access_key=obj_storage_secret_key,
                        config=boto3.session.Config(signature_version='s3v4'),
                        verify=False
                        )
    s3c.Bucket(minio_bucket).upload_file(file_path, minio_object_name)


In [7]:
# Upload file to minio
upload_file_to_minio('data.json', 'data', 'data-raw/data.json')
upload_file_to_minio('data2.json', 'data', 'data-raw/data2.json')
upload_file_to_minio('data3.json', 'data', 'data-raw/data3.json')